In [1]:
import numpy as np
import scipy as sp

## How to solve a linear system $Ax=b$?

## Inverse the matrix A

In [2]:
A = np.matrix([2., 3., 4., 9.]).reshape((2, 2))
A

matrix([[2., 3.],
        [4., 9.]])

In [3]:
b = np.matrix([6., 15.]).reshape((2, 1))
b

matrix([[ 6.],
        [15.]])

In [4]:
detA = np.linalg.det(A)
detA

6.0

In [5]:
invA = 1/detA * np.matrix([[9., -3.], [-4., 2.]])
invA

matrix([[ 1.5       , -0.5       ],
        [-0.66666667,  0.33333333]])

In [6]:
np.round(A @ invA)

matrix([[ 1., -0.],
        [ 0.,  1.]])

In [7]:
np.round(invA @ A)

matrix([[1., 0.],
        [0., 1.]])

In [8]:
x = invA @ b
x

matrix([[1.5],
        [1. ]])

In general, we can still compute the inverse of the matrix A 

In [9]:
A = np.array([2., 1., -1., -3., -1., 2., -2., 1., 2.]).reshape((3,3))
A

array([[ 2.,  1., -1.],
       [-3., -1.,  2.],
       [-2.,  1.,  2.]])

In [10]:
b = np.array([8, -11., -3.]).reshape((3, 1))
b

array([[  8.],
       [-11.],
       [ -3.]])

In [11]:
invA = np.linalg.inv(A)
invA

array([[ 4.,  3., -1.],
       [-2., -2.,  1.],
       [ 5.,  4., -1.]])

In [12]:
x = invA @ b
x

array([[ 2.],
       [ 3.],
       [-1.]])

### Matrix decomposition methods

## Forward and backward subtition

In [13]:
# Solve Ux=b where U is upper triangular
def backward_subt(U, b):
    m, n = U.shape
    assert(m == n)
    x = np.zeros(n)
    for i in reversed(range(n)):
        s = 0.
        for j in range(i, n):
            s += U[i,j] * x[j]
        x[i] = (b[i] - s) / U[i,i]
    return x

In [14]:
# Solve Lx=b where L is lower triangular
def forward_subt(L, b):
    m, n = L.shape
    assert(m == n)
    x = np.zeros(n)
    for i in range(n):
        s = 0.
        for j in range(0, i):
            s += L[i,j] * x[j]
        x[i] = (b[i] - s) / L[i,i]
    return x

## QR decomposition

Decompose $A = QR$ where $Q$ is orthogonal($Q^{-1}=Q^{T}$) and $R$ is upper triangular

In [15]:
m, n = A.shape
Q, R = np.linalg.qr(A)

In [16]:
Q

array([[-0.48507125,  0.41166465,  0.77151675],
       [ 0.72760688, -0.29939248,  0.6172134 ],
       [ 0.48507125,  0.86075337, -0.15430335]])

In [17]:
R

array([[-4.12310563, -0.72760688,  2.9104275 ],
       [ 0.        ,  1.5718105 ,  0.71105713],
       [ 0.        ,  0.        ,  0.15430335]])

from $Ax=b$, we have $QRx=b$, and $Rx=Q^Tb$

solve $z = Q^Tb$ using backward subtitution

In [18]:
Q.T @ b

array([[-13.33945938],
       [  4.00437436],
       [ -0.15430335]])

In [19]:
z = (Q.T @ b).flatten()
x = backward_subt(R, z)
x

array([ 2.,  3., -1.])

### LU decomposition

Decompose a matrix into a product $A = PLU$

and solve $PLU x= b$, which is $LUx = P^Tb$

- Solve $Lz = t$ using forward substitution where $z = Ux$ and $t = P^T b$
- Solve $Ux = z$ using backward subtitution

In [20]:
P, L, U = sp.linalg.lu(A)

In [21]:
t = (P.T @ b).flatten()
z = forward_subt(L, t)
z

array([-11.        ,   4.33333333,  -0.2       ])

In [22]:
x = backward_subt(U, z)
x

array([ 2.,  3., -1.])

### SVD decomposition

Decompose $A = U \Sigma V^T$

The solution is $x = V \Sigma^{-1} U^T b$

In [23]:
U, S, Vt = np.linalg.svd(A)

In [24]:
U

array([[-0.45463329, -0.45500496, -0.76568862],
       [ 0.72562679,  0.30930747, -0.61465002],
       [ 0.51650202, -0.83504454,  0.18954231]])

In [25]:
S

array([5.10343603, 1.71519373, 0.11424157])

In [26]:
Vt

array([[-0.80713287, -0.13006101,  0.57586514],
       [-0.09785674, -0.93246433, -0.34775615],
       [-0.58220322,  0.3370377 , -0.73989526]])

In [27]:
S

array([5.10343603, 1.71519373, 0.11424157])

In [28]:
invS = (np.ones(3) / S) 

In [29]:
def to_dense(x):
    n = x.shape[0]
    y = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            y[i,i] = x[i]
    return y

In [30]:
invS = np.ones(n) / S
invS

array([0.19594642, 0.58302452, 8.75338148])

In [31]:
x = Vt.T @ to_dense(invS) @ U.T @ b
x = x.flatten()
x

array([ 2.,  3., -1.])